## Loading in Data

In [ ]:
import types
import pandas as pd
import numpy as np
from botocore.client import Config
import ibm_boto3
from sklearn import preprocessing
import matplotlib.pyplot as plt
import requests # library to handle requests
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:

body = client_1ac46fe64048461aa02020f0a715fcf7.get_object(Bucket='thequotsimplequotchangeoftheunion-donotdelete-pr-bxu3clanfhxujd',Key='ACSST5Y2018.S0101_data_with_overlays_2020-04-15T174250.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

ACS_2018 = pd.read_csv(body)
ACS_2018.head()


In [ ]:

body = client_1ac46fe64048461aa02020f0a715fcf7.get_object(Bucket='thequotsimplequotchangeoftheunion-donotdelete-pr-bxu3clanfhxujd',Key='Poverty Data 2010.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

Poverty_2010 = pd.read_csv(body)
Poverty_2010.head()


In [ ]:

body = client_1ac46fe64048461aa02020f0a715fcf7.get_object(Bucket='thequotsimplequotchangeoftheunion-donotdelete-pr-bxu3clanfhxujd',Key='Poverty Data 2018.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

Poverty_2018 = pd.read_csv(body)
Poverty_2018.head()


In [ ]:
#Removing that top row of the ACS dataframes that are just description filler, not data

ACS_2018 = ACS_2018.iloc[1:]
ACS_2010 = ACS_2010.iloc[1:]

## Data Cleaning and Organizing

In [ ]:
#renaming columns in dfs

ACS_2018.rename(columns={'S0101_C01_001E': 'Total Population'}, inplace = True)
ACS_2018.rename(columns={'S0101_C03_001E': 'Total Male Population'}, inplace = True)
ACS_2018.rename(columns={'S0101_C05_001E': 'Total Female Population'}, inplace = True)

ACS_2010.rename(columns={'S0101_C01_001E': 'Total Population'}, inplace = True)
ACS_2010.rename(columns={'S0101_C02_001E': 'Total Male Population'}, inplace = True)
ACS_2010.rename(columns={'S0101_C03_001E': 'Total Female Population'}, inplace = True)


In [ ]:
#renaming columns in dfs

Poverty_2010.rename(columns={'All Ages in Poverty Percent': 'Poverty Percent'}, inplace = True)
Poverty_2010.rename(columns={'Median Household Income in Dollars': 'MHI'}, inplace = True)

Poverty_2018.rename(columns={'All Ages in Poverty Percent': 'Poverty Percent'}, inplace = True)
Poverty_2018.rename(columns={'Median Household Income in Dollars': 'MHI'}, inplace = True)

In [ ]:
#looking at two counties that have changed in the 10 year period: Shannon County SD, and Wade Hampton County AK
ACS_2010[ACS_2010['NAME'] == 'Shannon County, South Dakota']

In [ ]:
ACS_2018[ACS_2018['NAME'] == 'Oglala Lakota County, South Dakota'] #Shannon County was renamed Oglala Lakota County, and the GEO_ID has also changed

In [ ]:
ACS_2010[ACS_2010['NAME'] == 'Wade Hampton Census Area, Alaska']

In [ ]:
ACS_2018[ACS_2018['NAME'] == 'Kusilvak Census Area, Alaska'] #Wade Hamptom (2010) has been replaced with Kusilvak Census Area, also of a different geoid

#I kept the 2018 names, so we don't need to worry about changing the whole 2010 info. So we just need to change the GEO_ID 2010 version to the 2018 GEO_ID for these individual counties.

In [ ]:
Poverty_2010[Poverty_2010['State / County Name'] == 'Shannon County (SD)']

In [ ]:
Poverty_2018[Poverty_2018['State / County Name'] == 'Oglala Lakota County (SD)'] #ok, so 46102 is the county ID we need to switch the 2010 data to

In [ ]:
Poverty_2010.loc[Poverty_2010['State / County Name'] == 'Wade Hampton Census Area (AK)']

In [ ]:
Poverty_2018[Poverty_2018['State / County Name'] == 'Kusilvak Census Area (AK)'] #so we want the 2010 county id to be 2158

In [ ]:
#Pre-emptively converting those two counties 2010 GEOIDs to their future, 2018 ones. This way we can keep track of them as the same county in future datasets, even as their names change

#ACS Dataset

#first, changing Shannon County ID to its future County ID in 2018
ACS_2010.loc[ACS_2010['GEO_ID'] == '0500000US46113', 'GEO_ID'] = '0500000US46102'

#now, changing Wade Hampton Census Area County ID to its future County ID in 2018
ACS_2010.loc[ACS_2010['GEO_ID'] == '0500000US02270', 'GEO_ID']  = '0500000US02158'

In [ ]:
#Poverty Dataset

#first, changing Shannon County ID to its future County ID in 2018
Poverty_2010.loc[Poverty_2010['State / County Name'] == 'Shannon County (SD)', 'County ID'] = 46102

#now, changing Wade Hampton Census Area County ID to its future County ID in 2018
Poverty_2010.loc[Poverty_2010['State / County Name'] == 'Wade Hampton Census Area (AK)', 'County ID'] = 2158

In [ ]:
#ACS 2018 and 2010: Need to segment out their GEO Ids

ACS_2018['County ID'] = ACS_2018['GEO_ID'].str.slice(9)
ACS_2010['County ID'] = ACS_2010['GEO_ID'].str.slice(9)

In [ ]:
#Taking only the counties for the Poverty datasets

Poverty_2010 = Poverty_2010[Poverty_2010['State / County Name'].str.contains('\(',case= True)]
Poverty_2018 = Poverty_2018[Poverty_2018['State / County Name'].str.contains('\(',case= True)]

In [ ]:
Poverty_2018.head()

In [ ]:
#lets create a dataset with just the features we want
#we need to remember which data is from which year though
#check the meta-data files for a better look at which columsn correspond to what variable

#For ACS Features
Features_A2010 = ['GEO_ID', 'County ID', 'NAME', 'Total Population', 'Total Male Population', 'Total Female Population']
Features_A2018 = ['GEO_ID', 'County ID', 'NAME', 'Total Population', 'Total Male Population', 'Total Female Population']


#For Poverty Features
Features_P2010 = ['County ID', 'State / County Name', 'Poverty Percent', 'MHI']
Features_P2018 = ['County ID', 'State / County Name', 'Poverty Percent', 'MHI']

Ok, I just want to check that those feature pulls are working alright

In [ ]:
ACS_2018[Features_A2018].head() #cool, the Features_A2018 works

In [ ]:
ACS_2010[Features_A2010].head() #cool, the Features_A2010 works

In [ ]:
Poverty_2010[Features_P2010].head() #cool, the Features_P2010 works

In [ ]:
Poverty_2018[Features_P2018].head() #cool, the Features_P2018 works

Great, so I can fully segment these into new dataframes now

In [ ]:
dfP_2010 = Poverty_2010[Features_P2010]
dfP_2018 = Poverty_2018[Features_P2018]

dfA_2010 = ACS_2010[Features_A2010]
dfA_2018 = ACS_2018[Features_A2018]

In [ ]:
##Checking the datasets

#dfP_2010.head()
dfP_2018.head()
#dfA_2010.head()
#dfA_2018.head()

In [ ]:
#Splitting the 'County, State' format of the ACS datasets into two separate columns: County and State

splitCol = dfA_2018['NAME'].str.split(",", expand = True)
dfA_2018['County'] = splitCol[0]
dfA_2018['State'] = splitCol[1]
dfA_2018.drop(columns = ['NAME'], inplace = True)

splitCol = dfA_2010['NAME'].str.split(",", expand = True)
dfA_2010['County'] = splitCol[0]
dfA_2010['State'] = splitCol[1]
dfA_2010.drop(columns = ['NAME'], inplace = True)

In [ ]:
##Checking the datasets

#dfP_2010.head()
#dfP_2018.head()
dfA_2010.head()
#dfA_2018.head()

In [ ]:
#merge the Poverty to Poverty sets first, ACS to ACS. Both via the geo codes

#poverty 2010 to 2018 merge
dfP = pd.merge(left = dfP_2010, right = dfP_2018, on = 'County ID', suffixes = (' 2010', ' 2018'))

#ACS 2010 to 2018 merge
dfA = pd.merge(left = dfA_2010, right = dfA_2018, on = 'GEO_ID', suffixes = (' 2010', ' 2018'))

In [ ]:
dfP.head()

In [ ]:
dfA.head()

### Dealing with each dataset now, creating the change variables

**dfA** 
> needs change metrics between: Total population, Male population, Female population

In [ ]:
dfA['Total Population 2018'] = pd.to_numeric(dfA['Total Population 2018'])
dfA['Total Population 2010'] = pd.to_numeric(dfA['Total Population 2010'])
dfA['Total Male Population 2018'] = pd.to_numeric(dfA['Total Male Population 2018'])
dfA['Total Male Population 2010'] = pd.to_numeric(dfA['Total Male Population 2010'])
dfA['Total Female Population 2018'] = pd.to_numeric(dfA['Total Female Population 2018'])
dfA['Total Female Population 2010'] = pd.to_numeric(dfA['Total Female Population 2010'])

In [ ]:
dfA['% Total Population Change']  = round((dfA['Total Population 2018'] - dfA['Total Population 2010'])/(dfA['Total Population 2010'])*100, 2)
dfA['% Male Population Change']  = round((dfA['Total Male Population 2018'] - dfA['Total Male Population 2010'])/(dfA['Total Male Population 2010'])*100, 2)
dfA['% Female Population Change']  = round((dfA['Total Female Population 2018'] - dfA['Total Female Population 2010'])/(dfA['Total Female Population 2010'])*100, 2)

In [ ]:
dfA.head()

**dfP**
> needs change metrics between: MHI, Poverty

In [ ]:
#getting rid of the '$' and ','in MHI data
dfP['MHI 2010'] = dfP['MHI 2010'].str.slice(1)
dfP['MHI 2010'] = dfP['MHI 2010'].replace(',', '', regex = True)
dfP['MHI 2018'] = dfP['MHI 2018'].str.slice(1)
dfP['MHI 2018'] = dfP['MHI 2018'].replace(',', '', regex = True)

#changing to numerics for later calculations
dfP['Poverty Percent 2010'] = pd.to_numeric(dfP['Poverty Percent 2010'])
dfP['Poverty Percent 2018'] = pd.to_numeric(dfP['Poverty Percent 2018'])
dfP['MHI 2010'] = pd.to_numeric(dfP['MHI 2010'])
dfP['MHI 2018'] = pd.to_numeric(dfP['MHI 2018'])

In [ ]:
dfP['% Poverty Change'] = dfP['Poverty Percent 2018'] - dfP['Poverty Percent 2010']
dfP['% MHI Change'] = round((dfP['MHI 2018'] - dfP['MHI 2010'])/(dfP['MHI 2010'])*100, 2)

In [ ]:
dfP.head()

### Final Dataset

Merge the datasets, check that states and counties are lining up properly, then clean

In [ ]:
dfA['County ID'] = pd.to_numeric(dfA['County ID 2010'])

df = pd.merge(left = dfA, right = dfP, left_on = 'County ID', right_on = 'County ID')

In [ ]:
#just checking that the county 2010, state 2010, county 2018, state 2018, and State/county 2010 and 2018 are lining up
df.head()

In [ ]:
#Minor cleaning

df.drop(['State / County Name 2010', 'State / County Name 2018', 'County 2010', 'State 2010', 'County ID 2010'], axis = 1, inplace = True)

df.rename(columns = {'County 2018': 'County'}, inplace = True)
df.rename(columns = {'State 2018': 'State'}, inplace = True)

df['State'] = df['State'].str.lstrip()

In [ ]:
df.head()

In [ ]:
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

#forma to use the movecol function
#df = movecol(df, 
#             cols_to_move=['col1','col2'], 
#             ref_col='col3',
#             place='After' or 'Before')

In [ ]:
#moving some columns around
#is using a whole new function overkill here? Absolutely. But its good practice for me to use. This is not my function btw, credit to Adam Ross Nelson.

df = movecol(df, 
             cols_to_move = ['County ID', 'County', 'State'],
             ref_col = 'Total Population 2010',
             place = 'Before')

df = movecol(df, 
             cols_to_move = ['% Total Population Change', '% Male Population Change', '% Female Population Change'],
             ref_col = '% Poverty Change',
             place = 'Before')

df = movecol(df, 
             cols_to_move = ['Poverty Percent 2010', 'MHI 2010'],
             ref_col = 'Total Female Population 2010',
             place = 'After')

In [ ]:
df.head()

In [ ]:
df.describe()

# Analyses

Ok, now that I have the data I can do some analyses. I want to see clusters of:
> *what counties were similar in 2010?* -- using raw, standardized data \
> *what counties are similar now in 2018?* -- using raw, standardized data \
> *what counties changes in similar ways?* -- using the % change metrics

In [ ]:
df.describe()

Snapshot Analyses:

In [ ]:
#2010
Features_2010 = ['Total Population 2010','MHI 2010']

#2018
Features_2018 = ['Total Population 2018', 'MHI 2018']

#Delta
Features_delta = ['% Total Population Change', '% MHI Change']

In [ ]:
plt.scatter(df['Total Population 2010'], df['MHI 2010'])

In [ ]:
plt.scatter(df['Total Population 2018'], df['MHI 2018'])

Lets see what the min/max, stdev looks like without it those huge counties (E.g. LA). Then we can use those more descriptive stats and apply it to the original dataframe--in effect, dodging the outliers without removing its data

In [ ]:
df[df['Total Population 2010'] > 5000000] #so LA County and Cook County seem to be those big population centres

In [ ]:
df_Otl = df[(df.GEO_ID != '0500000US06037')] #Removing LA County
df_Otl = df_Otl[(df_Otl.GEO_ID != '0500000US17031')] #Removing Cook County
df_Otl

In [ ]:
df.describe()

In [ ]:
df_Otl.describe() #see the change in the population metrics?

In [ ]:
plt.scatter(df_Otl['Total Population 2010'], df_Otl['MHI 2010'])

Now, to quantile the upper, middle, and lower thirds for both population and mhi of this new outlier-less dataframe. We'll apply those parameters to the original df later

In [ ]:
Pop_parameters_2010 = df_Otl['Total Population 2010'].quantile([0, 0.33, 0.66, 1])
Pop_parameters_2010

In [ ]:
plt.scatter(df_Otl['Total Population 2010'], df_Otl['MHI 2010'])

plt.axvline(x=Pop_parameters_2010[.33])
plt.axvline(x=Pop_parameters_2010[.66])

So we can see that quantiles aren't super useful here because the data is so close. Lets see if a split across the range is clearer.

In [ ]:
print('Minimum Population =', df_Otl['Total Population 2010'].min(), 'Maximum Population = ', df_Otl['Total Population 2010'].max())

In [ ]:
pop10_third = (((df_Otl['Total Population 2010'].max()) - (df_Otl['Total Population 2010'].min()))/3)

In [ ]:
plt.scatter(df_Otl['Total Population 2010'], df_Otl['MHI 2010'])

plt.axvline(x= pop10_third)
plt.axvline(x= 2*pop10_third)

This seems to work out a bit better. Lets try the MHI too

In [ ]:
pop10_third = (((df_Otl['Total Population 2010'].max()) - (df_Otl['Total Population 2010'].min()))/3)
MHI10_third = (((df_Otl['MHI 2010'].max()) - (df_Otl['MHI 2010'].min()))/3)

plt.scatter(df_Otl['Total Population 2010'], df_Otl['MHI 2010'])

plt.axvline(x= pop10_third)
plt.axvline(x= 2*pop10_third)

plt.axhline(y= MHI10_third)
plt.axhline(y= 2*MHI10_third)

OK, so we have pop10_third and MHI10_third as decent markers for 2010 to apply to our full data later on. Lets quickly make the 2018 versions

In [ ]:
pop18_third = (((df_Otl['Total Population 2018'].max()) - (df_Otl['Total Population 2018'].min()))/3)
MHI18_third = (((df_Otl['MHI 2018'].max()) - (df_Otl['MHI 2018'].min()))/3)

plt.scatter(df_Otl['Total Population 2018'], df_Otl['MHI 2018'])

plt.axvline(x= pop18_third)
plt.axvline(x= 2*pop18_third)

plt.axhline(y= MHI18_third)
plt.axhline(y= 2*MHI18_third)

Iiiiiiii don't quite know what to do with the Change data yet. Its so generic (see graph) that I'm not convinced I can easily do this third-split with that data too. Lets hold on that for now.

In [ ]:
plt.scatter(df['% Total Population Change'], df['% MHI Change'])

In [ ]:
#adding in categorical variables to a new column based off of our outlier-less third ranges
def f(row):
    if row['Total Population 2010'] < pop10_third:
        val = 0
    elif row['Total Population 2010'] > 2*pop10_third:
        val = 2
    else:
        val = 1
    return val

df['2010 Population Split'] = df.apply(f, axis = 1)

In [ ]:
df['2010 Population Split'].value_counts()

In [ ]:
def f(row):
    if row['MHI 2010'] < MHI10_third:
        val = 0
    elif row['MHI 2010'] > 2*MHI10_third:
        val = 2
    else:
        val = 1
    return val

df['2010 MHI Split'] = df.apply(f, axis = 1)

In [ ]:
df['2010 MHI Split'].value_counts()

In [ ]:
def f(row):
    if row['2010 Population Split'] == 0 and row['2010 MHI Split'] == 0: #small, poor
        val = 0
    elif row['2010 Population Split'] == 1 and row['2010 MHI Split'] == 0: #medium, poor
        val = 1
    elif row['2010 Population Split'] == 2 and row['2010 MHI Split'] == 0: #large, poor
        val = 2
    elif row['2010 Population Split'] == 0 and row['2010 MHI Split'] == 1: #small, medium
        val = 3
    elif row['2010 Population Split'] == 1 and row['2010 MHI Split'] == 1: #medium, medium
        val = 4
    elif row['2010 Population Split'] == 2 and row['2010 MHI Split'] == 1: #large, medium
        val = 5
    elif row['2010 Population Split'] == 0 and row['2010 MHI Split'] == 2: #small, rich
        val = 6
    elif row['2010 Population Split'] == 1 and row['2010 MHI Split'] == 2: #medium, rich
        val = 7
    elif row['2010 Population Split'] == 2 and row['2010 MHI Split'] == 2: #large, rich
        val = 8
    else:
        val = 100
    return val
df['2010 Split'] = df.apply(f, axis = 1)

In [ ]:
df['2010 Split'].value_counts()

In [ ]:
def f(row):
    if row['Total Population 2018'] < pop18_third:
        val = 0
    elif row['Total Population 2018'] > 2*pop18_third:
        val = 2
    else:
        val = 1
    return val

df['2018 Population Split'] = df.apply(f, axis = 1)

In [ ]:
df['2018 Population Split'].value_counts()

In [ ]:
def f(row):
    if row['MHI 2018'] < MHI18_third:
        val = 0
    elif row['MHI 2018'] > 2*MHI18_third:
        val = 2
    else:
        val = 1
    return val

df['2018 MHI Split'] = df.apply(f, axis = 1)

In [ ]:
df['2018 MHI Split'].value_counts()

In [ ]:
def f(row):
    if row['2018 Population Split'] == 0 and row['2018 MHI Split'] == 0: #small, poor
        val = 0
    elif row['2018 Population Split'] == 1 and row['2018 MHI Split'] == 0: #medium, poor
        val = 1
    elif row['2018 Population Split'] == 2 and row['2018 MHI Split'] == 0: #large, poor
        val = 2
    elif row['2018 Population Split'] == 0 and row['2018 MHI Split'] == 1: #small, medium
        val = 3
    elif row['2018 Population Split'] == 1 and row['2018 MHI Split'] == 1: #medium, medium
        val = 4
    elif row['2018 Population Split'] == 2 and row['2018 MHI Split'] == 1: #large, medium
        val = 5
    elif row['2018 Population Split'] == 0 and row['2018 MHI Split'] == 2: #small, rich
        val = 6
    elif row['2018 Population Split'] == 1 and row['2018 MHI Split'] == 2: #medium, rich
        val = 7
    elif row['2018 Population Split'] == 2 and row['2018 MHI Split'] == 2: #large, rich
        val = 8
    else:
        val = 100
    return val
df['2018 Split'] = df.apply(f, axis = 1)

In [ ]:
df['2018 Split'].value_counts()

### DBSCAN Analyses

**2010 Clustering Snapshot**

In [ ]:
#Grabbing the features we want for the 2010 snapshot
X = df[Features_2010]

#taking out any NaNs, and standardizing the data.
X = np.nan_to_num(X)
X = preprocessing.StandardScaler().fit(X).transform(X)

In [ ]:
X

In [ ]:
#finding reasonable DBSCAN input values
neigh = NearestNeighbors(n_neighbors = 2)

nbrs = neigh.fit(X)

distances, indices = nbrs.kneighbors(X)

distances = np.sort(distances, axis = 0)

distances = distances[:, 1]

In [ ]:
plt.plot(distances) #our optimal value of epsilon is the point of maximum curvature
plt.ylim((0,1)) #looks like epsilon should be about 0.2

In [ ]:
from sklearn.cluster import DBSCAN
import sklearn.utils

sklearn.utils.check_random_state(1000)

#Compute DBSCAN
db = DBSCAN(eps = .1, min_samples = 4).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype = bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

In [ ]:
from scipy import stats
stats.describe(db.labels_)

In [ ]:
plt.hist(db.labels_)

In [ ]:
#insert the 2010 snapshot into the main df
df.insert(0, 'DBSCAN 2010 Cluster', db.labels_)

**2018 Snapshot Clustering**

In [ ]:
#Grabbing the features we want for the 2018 snapshot
X = df[Features_2018]

#taking out any NaNs, and standardizing the data.
X = np.nan_to_num(X)
X = preprocessing.StandardScaler().fit(X).transform(X)

In [ ]:
X

In [ ]:
#finding reasonable DBSCAN input values
neigh = NearestNeighbors(n_neighbors = 2)

nbrs = neigh.fit(X)

distances, indices = nbrs.kneighbors(X)

distances = np.sort(distances, axis = 0)

distances = distances[:, 1]

In [ ]:
plt.plot(distances) #our optimal value of epsilon is the point of maximum curvature
plt.ylim((0,1)) #looks like epsilon should be about 0.2

In [ ]:
from sklearn.cluster import DBSCAN
import sklearn.utils

sklearn.utils.check_random_state(1000)

#Compute DBSCAN
db = DBSCAN(eps = 0.2, min_samples = 5).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype = bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

realClusterNum=len(set(labels)) - (1 if -1 in labels else 0)
clusterNum = len(set(labels))

In [ ]:
#insert the 2018 snapshot into the main df
df.insert(0, 'DBSCAN 2018 Cluster', db.labels_)

**% Change Clustering**

In [ ]:
#Grabbing the features we want for the 2010 snapshot
X = df[Features_delta]

#taking out any NaNs, and standardizing the data.
X = np.nan_to_num(X)
X = preprocessing.StandardScaler().fit(X).transform(X)

In [ ]:
X

In [ ]:
#finding reasonable DBSCAN input values
neigh = NearestNeighbors(n_neighbors = 2)

nbrs = neigh.fit(X)

distances, indices = nbrs.kneighbors(X)

distances = np.sort(distances, axis = 0)

distances = distances[:, 1]

In [ ]:
plt.plot(distances) #our optimal value of epsilon is the point of maximum curvature
plt.ylim((0,1)) #looks like epsilon should be about 0.25

In [ ]:
from sklearn.cluster import DBSCAN
import sklearn.utils

sklearn.utils.check_random_state(1000)

#Compute DBSCAN
db = DBSCAN(eps = 0.3, min_samples = 5).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype = bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

realClusterNum=len(set(labels)) - (1 if -1 in labels else 0)
clusterNum = len(set(labels))

In [ ]:
#insert the change clustering into the main df
df.insert(0, 'DBSCAN Change Cluster', db.labels_)

In [ ]:
df['DBSCAN Change Cluster'].value_counts()

In [ ]:
df['DBSCAN 2018 Cluster'].value_counts()

In [ ]:
df['DBSCAN 2010 Cluster'].value_counts()

In [ ]:
df.head() #nice, all of our clusters are showing up in the dataframe

### KMeans Analyses

2010 Snapshot

In [ ]:
#Grabbing the features we want for the 2010 snapshot
X = df[Features_2010]

#taking out any NaNs, and standardizing the data.
X = np.nan_to_num(X)
X = preprocessing.StandardScaler().fit(X).transform(X)

#running iterative KMeans the analyses
for clusters in range (1, 12):
    kmeans = KMeans(n_clusters = clusters, random_state = 0).fit(X)
    unique, counts = np.unique(kmeans.labels_, return_counts = True)
    print(dict(zip(unique, counts)))

In [ ]:
KMeans(n_clusters = 9, random_state = 0).fit(X)

In [ ]:
#insert the 2010 clustering into the main df
df.insert(0, 'Kmeans 2010 Cluster', kmeans.labels_)

2018 Snapshot

In [ ]:
#Grabbing the features we want for the 2010 snapshot
X = df[Features_2018]

#taking out any NaNs, and standardizing the data.
X = np.nan_to_num(X)
X = preprocessing.StandardScaler().fit(X).transform(X)

#running iterative KMeans the analyses
for clusters in range (1, 12):
    kmeans = KMeans(n_clusters = clusters, random_state = 0).fit(X)
    unique, counts = np.unique(kmeans.labels_, return_counts = True)
    print(dict(zip(unique, counts)))

In [ ]:
KMeans(n_clusters = 9, random_state = 0).fit(X)

In [ ]:
#insert the 2018 clustering into the main df
df.insert(0, 'Kmeans 2018 Cluster', kmeans.labels_)

Change Clusters

In [ ]:
#Grabbing the features we want for the 2010 snapshot
X = df[Features_delta]

#taking out any NaNs, and standardizing the data.
X = np.nan_to_num(X)
X = preprocessing.StandardScaler().fit(X).transform(X)

#running iterative KMeans the analyses
for clusters in range (1, 12):
    kmeans = KMeans(n_clusters = clusters, random_state = 0).fit(X)
    unique, counts = np.unique(kmeans.labels_, return_counts = True)
    print(dict(zip(unique, counts)))

In [ ]:
KMeans(n_clusters = 9, random_state = 0).fit(X)

In [ ]:
#insert the 2010 clustering into the main df
df.insert(0, 'Kmeans Change Cluster', kmeans.labels_)

In [ ]:
df.describe() #nothing looks too crazy, which is good. We have 16 groups in Change and 2018 clusters, 21 for 2010 clusters

## Visualization

Showing the clusters--what characterizes each? What counties are in each?

**2010**

In [ ]:
df[['DBSCAN 2010 Cluster', 'Total Population 2010', 'Poverty Percent 2010', 'MHI 2010']].groupby(['DBSCAN 2010 Cluster']).mean()

In [ ]:
df[['Kmeans 2010 Cluster', 'Total Population 2010', 'Poverty Percent 2010', 'MHI 2010']].groupby(['Kmeans 2010 Cluster']).mean()

**2018**

In [ ]:
df[['DBSCAN 2018 Cluster', 'Total Population 2018', 'Poverty Percent 2018', 'MHI 2018']].groupby(['DBSCAN 2018 Cluster']).mean()

In [ ]:
df[['Kmeans 2018 Cluster', 'Total Population 2018', 'Poverty Percent 2018', 'MHI 2018']].groupby(['Kmeans 2018 Cluster']).mean()

**Change**

In [ ]:
df[['DBSCAN Change Cluster', '% Total Population Change', '% Poverty Change', '% MHI Change']].groupby(['DBSCAN Change Cluster']).mean()

In [ ]:
df[['Kmeans Change Cluster', '% Total Population Change', '% Poverty Change', '% MHI Change']].groupby(['Kmeans Change Cluster']).mean()

## Maps

In [ ]:
!pip install folium
import folium
import os
import json

In [ ]:
#creating a basic map of the USA

latitude = 37
longitude = -95

USA_map = folium.Map(location=[latitude, longitude], zoom_start = 4)
USA_map

In [ ]:
#download geojson file of all counties in USA. From my github--i converted that file from the

!wget --quiet https://github.com/hajnoszian/ForFutureReference/raw/master/cb_2018_us_county_5m.geojson

In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd
county_geo = 'cb_2018_us_county_5m.geojson'
geoPdata = gpd.read_file(county_geo)

In [ ]:
dfGEO = pd.merge(
    left = geoPdata,
    right = df,
    left_on = 'AFFGEOID', right_on = 'GEO_ID')
dfGEO.head()

In [ ]:
dfGEO.to_file('full_county_data.geojson', driver = 'GeoJSON')

In [ ]:
county_geo = 'full_county_data.geojson'

#to check the key on variables
#with open(county_geo) as f:
#    data = json.load(f)
#data 

This first one is a test to see if the counties map on correctly

In [ ]:
latitude = 37
longitude = -95

USA_map = folium.Map(location=[latitude, longitude], zoom_start = 4)
USA_map

#plotting the boundaries
folium.Choropleth(
    geo_data = county_geo,
    data = df,
    columns = ['GEO_ID', 'Total Population 2010'],
    key_on = 'feature.properties.AFFGEOID',
    fill_color = 'YlOrRd',
    fill_opacity = 0.6,
    line_opacity= .4,
    legend_name = "2010 Total Population"
).add_to(USA_map)
USA_map
#now to add descriptives

### Maps of 2010

> Population \
> MHI \
> Clustering \
> Split

**Population 2010**

In [ ]:
latitude = 37
longitude = -95

USA_map = folium.Map(location=[latitude, longitude], zoom_start = 4)

bins = list(df['Total Population 2010'].quantile([0, 0.1, 0.2, 0.3, 0.5, 0.7, 0.8, 0.9, 1])) #this is to break up the legend into more discernible color groupings


#plotting the boundaries
folium.Choropleth(
    geo_data = county_geo,
    data = df,
    columns = ['GEO_ID', 'Total Population 2010'],
    key_on = 'feature.properties.AFFGEOID',
    fill_color = 'Blues',
    fill_opacity = 0.6,
    line_opacity= .4,
    legend_name = "2010 Total Population",
    bins = bins,
).add_to(USA_map)
USA_map


**MHI 2010**

In [ ]:
latitude = 37
longitude = -95

USA_map = folium.Map(location=[latitude, longitude], zoom_start = 4)
USA_map

bins = list(df['MHI 2010'].quantile([0, 0.1, 0.2, 0.3, 0.5, 0.7, 0.8, 0.9, 1])) #this is to break up the legend into more discernible color groupings

#plotting the boundaries
folium.Choropleth(
    geo_data = county_geo,
    data = df,
    columns = ['GEO_ID', 'MHI 2010'],
    key_on = 'feature.properties.AFFGEOID',
    fill_color = 'BuGn',
    fill_opacity = 0.6,
    line_opacity= .4,
    legend_name = "MHI 2010",
    bins = bins
).add_to(USA_map)
USA_map

In [ ]:
df.head()

**Clusters 2010**

In [ ]:
latitude = 37
longitude = -95

USA_map = folium.Map(location=[latitude, longitude], zoom_start = 4)
USA_map

#plotting the boundaries
choropleth = folium.Choropleth(
    geo_data = county_geo,
    data = df,
    columns = ['GEO_ID', 'Kmeans 2010 Cluster'],
    key_on = 'feature.properties.AFFGEOID',
    fill_color = 'Set1',
    fill_opacity = 0.6,
    line_opacity= .4,
    highlight = True,
    legend_name = "Kmeans Clusters"
).add_to(USA_map)

#adding labels
style_function = "font-size: 15px; font-weight: bold"
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(fields = ['County', 'Kmeans 2010 Cluster', 'Total Population 2010', 'MHI 2010'],
                                   aliases = ['County Name', 'Cluster', 'Population', 'MHI'],
                                   labels = True))

USA_map

In [ ]:
latitude = 37
longitude = -95

USA_map = folium.Map(location=[latitude, longitude], zoom_start = 4)
USA_map

#plotting the boundaries
choropleth = folium.Choropleth(
    geo_data = county_geo,
    data = df,
    columns = ['GEO_ID', 'DBSCAN 2010 Cluster'],
    key_on = 'feature.properties.AFFGEOID',
    fill_color = 'Set1',
    fill_opacity = 0.6,
    line_opacity= .4,
    highlight = True,
    legend_name = "DBSCAN Clusters"
).add_to(USA_map)

#adding labels
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(fields = ['County', 'DBSCAN 2010 Cluster', 'Total Population 2010', 'MHI 2010'],
                                   aliases = ['County Name', 'Cluster', 'Population', 'MHI'],
                                   labels = True))

USA_map

### Maps of 2018

> Population \
> MHI \
> Clustering \
> Split

**Population 2018**

In [ ]:
latitude = 37
longitude = -95

USA_map = folium.Map(location=[latitude, longitude], zoom_start = 4)
USA_map

bins = list(df['Total Population 2018'].quantile([0, 0.2, 0.4, 0.6, 0.8, 1])) #this is to break up the legend into more discernible color groupings

#plotting the boundaries
folium.Choropleth(
    geo_data = county_geo,
    data = df,
    columns = ['GEO_ID', 'Total Population 2018'],
    key_on = 'feature.properties.AFFGEOID',
    fill_color = 'PuBu',
    fill_opacity = 0.6,
    line_opacity= .4,
    legend_name = "2018 Total Population",
    bins = bins
).add_to(USA_map)
USA_map

**MHI 2018**

In [ ]:
latitude = 37
longitude = -95

USA_map = folium.Map(location=[latitude, longitude], zoom_start = 4)
USA_map

bins = list(df['MHI 2018'].quantile([0, 0.2, 0.4, 0.6, 0.8, 1])) #this is to break up the legend into more discernible color groupings

#plotting the boundaries
folium.Choropleth(
    geo_data = county_geo,
    data = df,
    columns = ['GEO_ID', 'MHI 2018'],
    key_on = 'feature.properties.AFFGEOID',
    fill_color = 'BuGn',
    fill_opacity = 0.6,
    line_opacity= .4,
    legend_name = "MHI 2018",
    bins = bins
).add_to(USA_map)
USA_map

**Clusters 2018**

In [ ]:
latitude = 37
longitude = -95

USA_map = folium.Map(location=[latitude, longitude], zoom_start = 4)
USA_map

#plotting the boundaries
choropleth = folium.Choropleth(
    geo_data = county_geo,
    data = df,
    columns = ['GEO_ID', 'Kmeans 2018 Cluster'],
    key_on = 'feature.properties.AFFGEOID',
    fill_color = 'Set1',
    fill_opacity = 0.6,
    line_opacity= .4,
    highlight = True,
    legend_name = "Kmeans Clusters"
).add_to(USA_map)

#adding labels

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(fields = ['County', 'Kmeans 2018 Cluster', 'Total Population 2018', 'MHI 2018'],
                                   aliases = ['County Name', 'Cluster', 'Population', 'MHI'],
                                   labels = True))

USA_map

In [ ]:
latitude = 37
longitude = -95

USA_map = folium.Map(location=[latitude, longitude], zoom_start = 4)
USA_map

#plotting the boundaries
choropleth = folium.Choropleth(
    geo_data = county_geo,
    data = df,
    columns = ['GEO_ID', 'DBSCAN 2018 Cluster'],
    key_on = 'feature.properties.AFFGEOID',
    fill_color = 'Set1',
    fill_opacity = 0.6,
    line_opacity= .4,
    highlight = True,
    legend_name = "DBSCAN Clusters"
).add_to(USA_map)

#adding labels
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(fields = ['County', 'DBSCAN 2018 Cluster', 'Total Population 2018', 'MHI 2018'],
                                   aliases = ['County Name', 'Cluster', 'Population', 'MHI'],
                                   labels = True))

USA_map

### Change from 2010 to 2018

In [ ]:
df.head()

**Population Change**

In [ ]:
latitude = 37
longitude = -95

USA_map = folium.Map(location=[latitude, longitude], zoom_start = 4)
USA_map

bins = list(df['% Total Population Change'].quantile([0, 0.2, 0.4, 0.6, 0.8, 1])) #this is to break up the legend into more discernible color groupings


#plotting the boundaries
choropleth = folium.Choropleth(
    geo_data = county_geo,
    data = df,
    columns = ['GEO_ID', '% Total Population Change'],
    key_on = 'feature.properties.AFFGEOID',
    fill_color = 'RdBu',
    fill_opacity = 0.6,
    line_opacity= .4,
    highlight = True,
    legend_name = "DBSCAN Clusters"
).add_to(USA_map)

#adding labels
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(fields = ['County', 'Total Population 2010', 'Total Population 2018','% Total Population Change'],
                                   aliases = ['County Name', '2010 Population', '2018 Population', 'Change'],
                                   labels = True))

USA_map

**MHI Change**

In [ ]:

latitude = 37
longitude = -95

USA_map = folium.Map(location=[latitude, longitude], zoom_start = 4)
USA_map

bins = list(df['% MHI Change'].quantile([0, 0.2, 0.4, 0.6, 0.8, 1])) #this is to break up the legend into more discernible color groupings

#plotting the boundaries
folium.Choropleth(
    geo_data = county_geo,
    data = df,
    columns = ['GEO_ID', '% MHI Change'],
    key_on = 'feature.properties.AFFGEOID',
    fill_color = 'RdBu',
    fill_opacity = 0.6,
    line_opacity= .4,
    legend_name = "% MHI Change",
    bins = bins
).add_to(USA_map)

#adding labels
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(fields = ['County', 'MHI 2010', 'MHI 2018', '% MHI Change'],
                                   aliases = ['County Name', 'MHI 2010', 'MHI 2018', 'Change'],
                                   labels = True))

USA_map

**Poverty Change**

In [ ]:
latitude = 37
longitude = -95

USA_map = folium.Map(location=[latitude, longitude], zoom_start = 4)
USA_map

bins = list(df['% Poverty Change'].quantile([0, 0.2, 0.4, 0.6, 0.8, 1])) #this is to break up the legend into more discernible color groupings

#plotting the boundaries
folium.Choropleth(
    geo_data = county_geo,
    data = df,
    columns = ['GEO_ID', '% Poverty Change'],
    key_on = 'feature.properties.AFFGEOID',
    fill_color = 'RdBu',
    fill_opacity = 0.6,
    line_opacity= .4,
    legend_name = "% Poverty Change",
    bins = bins
).add_to(USA_map)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(fields = ['County', 'MHI 2010', 'MHI 2018', '% Poverty Change'],
                                   aliases = ['County Name', 'MHI 2010', 'MHI 2018', 'Change'],
                                   labels = True))
USA_map